# Computing performance evaluation metrics
Get some model predictions that we can then compute performance metrics for.

## 1. Load a trained model
To save time we will load a previously trained model.

In [ ]:
from xai.models.simple_cnn import CNNClassifier

## 2. Get model predictions

## 3. Calculate performance metrics
- accuracy
- auc
- f1

In [1]:
from sklearn.metrics import accuracy_score, auc, f1_score

In [2]:
accuracy_score?

Signature: accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
Docstring:
Accuracy classification score.

In multilabel classification, this function computes subset accuracy:
the set of labels predicted for a sample must *exactly* match the
corresponding set of labels in y_true.

Read more in the :ref:`User Guide <accuracy_score>`.

Parameters
----------
y_true : 1d array-like, or label indicator array / sparse matrix
    Ground truth (correct) labels.

y_pred : 1d array-like, or label indicator array / sparse matrix
    Predicted labels, as returned by a classifier.

normalize : bool, default=True
    If ``False``, return the number of correctly classified samples.
    Otherwise, return the fraction of correctly classified samples.

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

Returns
-------
score : float
    If ``normalize == True``, return the fraction of correctly
    classified samples (float), else returns the number of c